# DS-SF-36 | 10 | Linear Regression, Part 3 | Assignment | Starter Code

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

from sklearn import feature_selection, linear_model

In [2]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-10-credit.csv'))

In [3]:
df

,Income,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,14.891,283,2,34,11,Male,No,Yes,Caucasian,333
1,106.025,483,3,82,15,Female,Yes,Yes,Asian,903
2,104.593,514,4,71,11,Male,No,No,Asian,580
3,148.924,681,3,36,11,Female,No,No,Asian,964
4,55.882,357,2,68,16,Male,No,Yes,Caucasian,331
...,...,...,...,...,...,...,...,...,...,...
395,12.096,307,3,32,13,Male,No,Yes,Caucasian,560
396,13.364,296,5,65,17,Male,No,No,African American,480
397,57.872,321,5,67,12,Female,No,Yes,Caucasian,138
398,37.728,192,1,44,13,Male,No,Yes,Caucasian,0


In [25]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Income,400.0,45.218885,35.244273,10.354,21.00725,33.1155,57.47075,186.634
Rating,400.0,354.940000,154.724143,93.000,247.25000,344.0000,437.25000,982.000
Cards,400.0,2.957500,1.371275,1.000,2.00000,3.0000,4.00000,9.000
Age,400.0,55.667500,17.249807,23.000,41.75000,56.0000,70.00000,98.000
Education,400.0,13.450000,3.125207,5.000,11.00000,14.0000,16.00000,20.000
...,...,...,...,...,...,...,...,...
Ethnicity_Caucasian,400.0,0.497500,0.500620,0.000,0.00000,0.0000,1.00000,1.000
Gender_Male,400.0,0.482500,0.500319,0.000,0.00000,0.0000,1.00000,1.000
Married_Yes,400.0,0.612500,0.487790,0.000,0.00000,1.0000,1.00000,1.000
Student_Yes,400.0,0.100000,0.300376,0.000,0.00000,0.0000,0.00000,1.000


A description of the dataset is as follows:

- Income (in thousands of dollars)
- Rating: Credit score rating
- Cards: Number of credit cards owned
- Age
- Education: Years of education
- Gender: Male/Female
- Student: Yes/No
- Married: Yes/No
- Ethnicity: African American/Asian/Caucasian
- Balance: Average credit card debt

> ## Question 1.  Let's explore the quantitative variables that affect `Balance`.  From your preliminary analysis, which 2 variables seem to affect `Balance` the most?  Our goal is interpretation; can we use these 2 variables simultaneously?  Why or why not?

In [4]:
# TODO
df.corr()

,Income,Rating,Cards,Age,Education,Balance
Income,1.000000,0.791378,-0.018273,0.175338,-0.027692,0.463656
Rating,0.791378,1.000000,0.053239,0.103165,-0.030136,0.863625
Cards,-0.018273,0.053239,1.000000,0.042948,-0.051084,0.086456
Age,0.175338,0.103165,0.042948,1.000000,0.003619,0.001835
Education,-0.027692,-0.030136,-0.051084,0.003619,1.000000,-0.008062
Balance,0.463656,0.863625,0.086456,0.001835,-0.008062,1.000000


Answer: Rating and income are highly correlated to Balance, at 46% and 86%, respectively.  Rating and income are also very highly correlated at 79%.  There is a good chance the these two variables will cause colinearity.  

In [6]:
df.isnull().sum()

Income       0
Rating       0
Cards        0
Age          0
Education    0
Gender       0
Student      0
Married      0
Ethnicity    0
Balance      0
dtype: int64

> ## Question 2.  `Ethnicity`, `Gender`, `Married`, and `Student` are categorical variables.  Go ahead and create dummy variables for all of them.

In [5]:
# TODO
Ethnicity = pd.get_dummies(df.Ethnicity,prefix = 'Ethnicity', drop_first=True)
Ethnicity.head()

,Ethnicity_Asian,Ethnicity_Caucasian
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [7]:
Gender = pd.get_dummies(df.Gender,prefix = 'Gender', drop_first=True)
Gender.head()

,Gender_Male
0,1
1,0
2,1
3,0
4,1


In [8]:
Married = pd.get_dummies(df.Married,prefix = 'Married', drop_first=True)
Married.head()

,Married_Yes
0,1
1,1
2,0
3,0
4,1


In [9]:
Student = pd.get_dummies(df.Student,prefix = 'Student', drop_first=True)
Student.head()

,Student_Yes
0,0
1,1
2,0
3,0
4,0


In [10]:
df = pd.merge(df, Ethnicity,left_index=True, right_index=True)
df = pd.merge(df, Gender, left_index=True, right_index=True )
df = pd.merge(df, Married, left_index=True, right_index=True )
df = pd.merge(df, Student, left_index=True, right_index=True )
df.head()

,Income,Rating,Cards,Age,Education,...,Ethnicity_Asian,Ethnicity_Caucasian,Gender_Male,Married_Yes,Student_Yes
0,14.891,283,2,34,11,...,0,1,1,1,0
1,106.025,483,3,82,15,...,1,0,0,1,1
2,104.593,514,4,71,11,...,1,0,1,0,0
3,148.924,681,3,36,11,...,1,0,0,0,0
4,55.882,357,2,68,16,...,0,1,1,1,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 15 columns):
Income                 400 non-null float64
Rating                 400 non-null int64
Cards                  400 non-null int64
Age                    400 non-null int64
Education              400 non-null int64
Gender                 400 non-null object
Student                400 non-null object
Married                400 non-null object
Ethnicity              400 non-null object
Balance                400 non-null int64
Ethnicity_Asian        400 non-null uint8
Ethnicity_Caucasian    400 non-null uint8
Gender_Male            400 non-null uint8
Married_Yes            400 non-null uint8
Student_Yes            400 non-null uint8
dtypes: float64(1), int64(5), object(4), uint8(5)
memory usage: 33.3+ KB


> ## Question 3.  Using _sklearn_'s linear regression, predict `Balance` using `Income`, `Cards`, `Age`, `Education`, `Gender`, and `Ethnicity`

First, find the coefficients of your regression line.

In [12]:
# TODO
X = df[ ['Income','Cards','Age','Education','Gender_Male', \
         'Ethnicity_Asian','Ethnicity_Caucasian'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)

Then, find the p-values of your F-values' models.  You have a few variables, so try to show your p-values alongside the names of the variables.  (https://docs.python.org/2/library/functions.html#zip)

In [13]:
# TODO
def summary(X, y, model):
    _, f_pvalues = feature_selection.f_regression(X, y)

    print 'R^2 =', model.score(X, y)
    print

    print 'Coefficients'
    print '- beta_0 (Intercept) = {}'.format(model.intercept_)

    for i, coef in enumerate(model.coef_):
        print '- beta_{} ({}) = {} (p-value = {})'.format(i + 1, X.columns[i], coef, f_pvalues[i])

        
summary(X, y, model)

R^2 = 0.232312608335

Coefficients
- beta_0 (Intercept) = 257.167785624
- beta_1 (Income) = 6.27995894353 (p-value = 1.03088580259e-22)
- beta_2 (Cards) = 33.6295350792 (p-value = 0.0841765555994)
- beta_3 (Age) = -2.32970547308 (p-value = 0.97081387233)
- beta_4 (Education) = 1.64553607303 (p-value = 0.872306401567)
- beta_5 (Gender_Male) = -27.1254312316 (p-value = 0.668516105503)
- beta_6 (Ethnicity_Asian) = -6.546030781 (p-value = 0.844895644362)
- beta_7 (Ethnicity_Caucasian) = 3.47497641303 (p-value = 0.947727511397)


> ## Question 4.  Which of your coefficients are significant at the 5% significance level?

Answer: Income is significant at 5% significance level.

> ## Question 5.  What is your model's $R^2$?

In [14]:
# TODO
model.score(X,y)

0.23231260833540465

> ## Question 6.  How do we interpret this value?

Answer: 23% of the total variance in Balance can be explained by the model.

> ## Question 7.  Now let's focus on the two most significant variables from your previous model and re-run your regression model.

In [17]:
# TODO
X = df[ ['Income','Cards'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y, model)


R^2 = 0.223991751622

Coefficients
- beta_0 (Intercept) = 151.329946349
- beta_1 (Income) = 6.07099859467 (p-value = 1.03088580259e-22)
- beta_2 (Cards) = 31.8381289478 (p-value = 0.0841765555994)


> ## Question 8.  In comparison to the previous model, did the $R^2$ increase or decrease?  Why?

In [16]:
# TODO
model.score(X,y)

0.22399175162249518

Answer: R square decreased slightly from .23 to .22.  The more variables you have in the model, the higher R square will be.  When we decreased the model from previous version to just 2 variables, the R square decreases.

> ## Question 9.  Now let's regress `Balance` on `Gender` alone.  After running your linear regressions, do you have enough evidence to claim that females have more balance than males?  (Hint: Look at the p-value of the Gender coefficient.  If it is significant then you will have evidence to support that claim, otherwise you cannot support the statement.)

In [18]:
# TODO
X = df[ ['Gender_Male'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)


R^2 = 0.000461132964496

Coefficients
- beta_0 (Intercept) = 529.536231884
- beta_1 (Gender_Male) = -19.7331230758 (p-value = 0.668516105503)


Answer: (If your answer is yes, interpret the results).  We don't have enough evidence to claim that females have more balance than males.

> ## Question 10.  Now let's regress `Balance` on `Ethnicity`.  After running your linear regressions, do you have enough evidence to claim that some ethnic groups carry more balance than others?

In [19]:
# TODO
X = df[ ['Ethnicity_Asian','Ethnicity_Caucasian'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)

R^2 = 0.000218807443049

Coefficients
- beta_0 (Intercept) = 531.0
- beta_1 (Ethnicity_Asian) = -18.6862745098 (p-value = 0.844895644362)
- beta_2 (Ethnicity_Caucasian) = -12.5025125628 (p-value = 0.947727511397)


Answer: (If your answer is yes, interpret the results).  There is no evidence to claim that some ethnic groups carry more balance than others.

> ## Question 11.  Finally let's regress `Balance` on `Student`.  After running your linear regressions, do you have enough evidence to claim that students carry more balance than non-students?

In [20]:
# TODO
X = df[ ['Student_Yes'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)

R^2 = 0.0670900898871

Coefficients
- beta_0 (Intercept) = 480.369444444
- beta_1 (Student_Yes) = 396.455555556 (p-value = 1.48773410773e-07)


Answer: (If your answer is yes, interpret the results).  Yes, student carries on average $396 more in balance than non-students.

> ## Question 12.  No let's consider the effect of `Student` and `Income` on `Balance` simultaneously.  Are all the coefficients significant?

In [21]:
# TODO
X = df[ ['Student_Yes', 'Income'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)

R^2 = 0.277458888967

Coefficients
- beta_0 (Intercept) = 211.142964398
- beta_1 (Student_Yes) = 382.670538843 (p-value = 1.48773410773e-07)
- beta_2 (Income) = 5.98433556506 (p-value = 1.03088580259e-22)


Answer: (If your answer is yes, interpret the results).  Yes, all the coefficients are significant at 95% confidence.

> ## Question 13.  No let's consider the interaction effect of `Student` and `Income` on `Balance` simultaneously.  Are all the coefficients significant?  It they are, write down your regression model below

(First generate a new variable for the interaction term)

In [22]:
# TODO
df['Student_Income_interact']=df.Student_Yes * df.Income
df

,Income,Rating,Cards,Age,Education,...,Ethnicity_Caucasian,Gender_Male,Married_Yes,Student_Yes,Student_Income_interact
0,14.891,283,2,34,11,...,1,1,1,0,0.000
1,106.025,483,3,82,15,...,0,0,1,1,106.025
2,104.593,514,4,71,11,...,0,1,0,0,0.000
3,148.924,681,3,36,11,...,0,0,0,0,0.000
4,55.882,357,2,68,16,...,1,1,1,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,307,3,32,13,...,1,1,1,0,0.000
396,13.364,296,5,65,17,...,0,1,0,0,0.000
397,57.872,321,5,67,12,...,1,0,1,0,0.000
398,37.728,192,1,44,13,...,1,1,1,0,0.000


In [23]:
X = df[ ['Student_Yes', 'Income', 'Student_Income_interact'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)

R^2 = 0.279883703062

Coefficients
- beta_0 (Intercept) = 200.62315295
- beta_1 (Student_Yes) = 476.675843207 (p-value = 1.48773410773e-07)
- beta_2 (Income) = 6.21816873695 (p-value = 1.03088580259e-22)
- beta_3 (Student_Income_interact) = -1.99915087151 (p-value = 4.61768367757e-08)


Answer: Yes, all the variables are significant, including the interaction term.

> ## Question 14.  Is there any income level at which students and non-students on average carry same level of balance?

Answer: 200.62 + 476.68 + 6.22*Income - 2*Income =200.62 + 0 + 6.21*Income - 0.  Solve for Income gives us 476.68/2=238.34.